# Assignment 1.4 Pandas
Pandas is an open source Python package providing fast, flexible, and expressive data structures and data manipulation designed to make working with “relational” or “labeled” data both easy and intuitive. It is the fundamental high-level building block for doing practical, real world data analysis in Python.
https://pandas.pydata.org/

In [1]:
## Import numpy and pandas

# YOUR CODE HERE
import pandas as pd
import numpy as np

The two primary data structures of pandas are 
- Series (1-dimensional) and 
- DataFrame (2-dimensional)

Read this [10 min intro to get started](https://pandas.pydata.org/pandas-docs/stable/10min.html)

### Data Loading

You are given two csv files.  
- 'sales1.csv'
- 'sales2.csv'

Load both CSV as dataframes and Investigate the data and its types

In [4]:
# YOUR CODE HERE
sales1 = pd.read_csv("sales1.csv")
sales2 = pd.read_csv("sales2.csv")

,Item Type,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost
0,Cosmetics,10/18/2014,686800706,10/31/2014,8446.0,437.20,263.33
1,Vegetables,11/7/2011,185941302,12/8/2011,NaN,NaN,NaN
2,Baby Food,10/31/2016,246222341,12/9/2016,1517.0,255.28,159.42
3,*Cereal,4/10/2010,161442649,5/12/2010,3322.0,205.70,117.11
4,Fruits,8/16/2011,645713555,8/31/2011,9845.0,9.33,6.92


In [3]:
# Investigate the data 
# YOUR CODE HERE
print(sales1)
print(sales2)
print(sales1.describe())
print(sales2.describe())

           Item Type  Order Date   Order ID   Ship Date  Units Sold  \
0          Cosmetics  10/18/2014  686800706  10/31/2014      8446.0   
1         Vegetables   11/7/2011  185941302   12/8/2011         NaN   
2          Baby Food  10/31/2016  246222341   12/9/2016      1517.0   
3            *Cereal   4/10/2010  161442649   5/12/2010      3322.0   
4             Fruits   8/16/2011  645713555   8/31/2011      9845.0   
5            *Cereal  11/24/2014  683458888  12/28/2014      9528.0   
6            *Cereal    3/4/2015  679414975   4/17/2015      2844.0   
7            Clothes   5/17/2012  208630645   6/28/2012      7299.0   
8         Vegetables   1/29/2015  266467225    3/7/2015      2428.0   
9         Vegetables  12/24/2013  118598544   1/19/2014      4800.0   
10           Clothes  12/29/2015  451010930   1/19/2016      3012.0   
11            Snacks   2/27/2010  220003211   3/18/2010      2694.0   
12         Household  11/17/2016  702186715  12/22/2016      1508.0   
13    

### Merge Dataframes
Merge the two dataframes into a single dataframe "sales" based on order id such that new data consist of common order ID and all the other columns.
Also analyse the data in different columns.
Use [Merge function](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.merge.html)

In [6]:
sales = None
# YOUR CODE HERE
sales = pd.merge(sales1, sales2,on="Order ID", how="outer")

In [7]:
assert(sales[sales["Order ID"]==686800706]["Country"].values == "Libya")
assert(sales[sales["Order ID"]==686800706]["Item Type"].values == "Cosmetics")

### Data Cleaning
If you look at the data we can see some issues,
- Some of the Item Type start with " * " we want to remove the asterik in the column
- Some columns data are missing indicated by NaN (u can try isnull(), isna() function)
  - If Region or Country data is missing replace it with unknown
  - if other data are missing, remove it from "sales" dataframe and add it in a "missing" dataframe 

Reference for this section:
- [Working with missing data](https://pandas.pydata.org/pandas-docs/stable/missing_data.html)
- [Python Lambda Expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions)
- [Pandas Apply Function](https://pandas.pydata.org/pandas-docs/version/0.18/generated/pandas.Series.apply.html)
- [Pandas FillNa](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.fillna.html)

In [9]:
# Investigate dataframe for Null (NA/NAN) and count the number of rows with NA/NAN
NA_count = 0
# YOUR CODE HERE
NA_count = sales[sales.isnull().any(axis=1)].shape[0]


31

In [10]:
# remove "*", NaN, add missing sales data to "missing" dataframe 
# YOUR CODE HERE
sales["Item Type"]=pd.Series(sales["Item Type"]).str.replace("*","",regex=True)
sales["Country"].fillna("unknown", inplace=True)
sales["Region"].fillna("unknown", inplace=True)
missing = pd.DataFrame(columns=sales.columns)
for index,row in sales.iterrows():
    if row.isna().sum():
        missing = missing.append(sales.loc[index, :], ignore_index=True)
        sales.drop(index, inplace=True)

In [11]:
assert(sales[sales["Order ID"]==267066323]["Item Type"].values == "Cereal")
assert(sales[sales["Order ID"]==267066323]["Region"].values == "unknown")

Find the number of unique items available.

In [12]:
NumItems = 0
# YOUR CODE HERE
NumItems = len(pd.unique(sales["Item Type"]))

### Grouping and Analysis 
Find out the statistical summary for ‘Unit Cost’ column for each Items.
Then extract the count, mean and std for "Cereal","Fruits","Vegetables","Baby Food" and "Meat" as a dataframe "Kitchen"
References:
- [Pandas Indexing](https://pandas.pydata.org/pandas-docs/stable/indexing.html)
- [Groupby](https://pandas.pydata.org/pandas-docs/stable/groupby.html)
- [Loc](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.loc.html)
- [filter](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.filter.html)

In [13]:
Kitchen = None
# YOUR CODE HERE
indexdict = ["Cereal", "Fruits", "Vegetables", "Baby Food", "Meat"]
Kitchen = pd.DataFrame(index= indexdict, columns=["count", "mean", "std"])
for i in indexdict:
    unitCostCol = sales.loc[sales["Item Type"]== i ]["Unit Cost"]
    Kitchen.loc[ i ] = [unitCostCol.count(), unitCostCol.mean(), unitCostCol.std()]

In [14]:
assert(Kitchen.loc["Cereal"]["count"]==77)

In [15]:
#Generate the new column ‘Total Profit’ for each orders (Total Profit = Total Revenue - Total Cost)
# YOUR CODE HERE
sales.insert(12, "Total Profit", value= sales["Total Revenue"] - sales["Total Cost"])

In [16]:
assert(sales[sales["Order ID"]==246222341]["Total Profit"].values == 145419.62)

In [17]:
# How many data sets are there with profit greater than 60%
ans = None
# YOUR CODE HERE
sales["Total Profit%"] = sales["Total Profit"]/sales["Total Cost"]*100
ans = (sales["Total Profit%"]>=60).count()